In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
# Import the FOOOF object
from fooof import FOOOF
from fooof import FOOOFGroup
# Import a utility to download and load example data
from fooof.utils.download import load_fooof_data
from pathlib import Path
import glob
from math import floor


In [77]:
# get the staging list
path_stage = Path('/home/b1044271/EEGsleep/SleepStaging/mat/mne2/')
stage_files = os.listdir(path_stage)
stage_files = [file for file in stage_files]
stage_files = sorted(stage_files)

# get all stages files
path_stage2 = '/home/b1044271/EEGsleep/SleepStaging/*.txt'
stage_files2 = glob.glob(path_stage2)

# get the subjects list
Data_path = '/home/b1044271/Columbia/Preprocessed/Better_ica/'
subj_files = os.listdir(Data_path)
subj_files = [file for file in subj_files]
subj_files = sorted(subj_files)

# DUMMY
Dummy_path = '/home/b1044271/Columbia/Dummy'
dummy_files = glob.glob(Dummy_path)

# Create path for results
path_results = Path('/home/b1044271/Columbia/Results/Time-resolved')

In [3]:
# SETTINGS FOR FOOOF and PSD CALCULATIONS

t = 5 #in seconds
electrode = 'E257'
fs = 250 
# Settings for Welch's PSD analyses
SETTINGS_B  = { 'method' : 'welch', 'average' : 'mean', 'fmin' :1 , 'fmax':45, 'n_fft': fs*t, 'n_overlap': fs*t*0.5}

#FOOOF settings
SETTINGS_F1={'max_n_peaks':8, 'aperiodic_mode':'fixed'}
SETTINGS_F2={'max_n_peaks':8, 'aperiodic_mode':'knee'}

fm1 = FOOOFGroup(**SETTINGS_F1)
fm2 = FOOOFGroup(**SETTINGS_F2)

In [4]:
# START
EEG = mne.io.read_raw_eeglab(os.path.join(Data_path, subj_files[10])) # read raw .set file
EEG1 = EEG.pick(electrode, exclude=[]) # select Cz for further analysis

/home/b1044271/anaconda3/envs/mne/lib/python3.10/site-packages/pymatreader/utils.py:122: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/tmp/ipykernel_442224/198683572.py:2: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(os.path.join(Data_path, subj_files[10])) # read raw .set file
/tmp/ipykernel_442224/198683572.py:2: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  EEG = mne.io.read_raw_eeglab(os.path.join(Data_path, subj_files[10])) # read raw .set file


In [60]:
events = mne.read_events(Path(path_stage, stage_files[10])) #read staging markers
epochs = mne.Epochs(EEG1, events=events[2:-1], tmin=-60, tmax=0, preload=True)

events2 = mne.read_events(Path(Dummy_path, dummy_files[10])) #read staging markers


Not setting metadata
628 matching events found
Setting baseline interval to [-60.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 628 events and 15001 original time points ...
0 bad epochs dropped


/tmp/ipykernel_442224/2263904632.py:1: RuntimeWarning: This filename (/home/b1044271/EEGsleep/SleepStaging/mat/mne2/VP13_SS_250_mne.txt) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(Path(path_stage, stage_files[10])) #read staging markers


In [61]:
files  = pd.read_csv(stage_files2[10])
stages = list(files.iloc[:,0])

N2_o = [i for i in range(len(stages)) if stages[i] == 2]
N2_l = np.array(stages)[np.array([i + 1 for i in N2_o[0:-2]])]

N2_N3 = [i for i in range(len(N2_l)) if N2_l[i] == 3]
N2_N1 = [i for i in range(len(N2_l)) if N2_l[i] == 1]

In [62]:
T_N2N3 = epochs[[i for i in N2_N3]]
T_N2N1 = epochs[[i for i in N2_N3]]


In [73]:
X = T_N2N1.copy().crop(tmin=0, tmax=5)
X2 = T_N2N1.copy().crop(tmin=1, tmax=6)


/tmp/ipykernel_442224/4099869945.py:1: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.Epochs'>.tmax (0 sec)
  X = T_N2N1.copy().crop(tmin=0, tmax=5)
/tmp/ipykernel_442224/4099869945.py:2: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.Epochs'>.tmax (0 sec)
  X2 = T_N2N1.copy().crop(tmin=1, tmax=6)


ValueError: tmin (1) must be less than or equal to tmax (0.0)

In [76]:
X2 = T_N2N1.copy().crop(tmin=0, tmax=6)
X3 = mne.concatenate_epochs([X,X2], add_offset = False)


/tmp/ipykernel_442224/2979078062.py:1: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.Epochs'>.tmax (0 sec)
  X2 = T_N2N1.copy().crop(tmin=0, tmax=6)
/tmp/ipykernel_442224/2979078062.py:2: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X3 = mne.concatenate_epochs([X,X2], add_offset = False)


RuntimeError: Event time samples were not unique. Consider setting the `event_repeated` parameter."

In [66]:
X.__dict__

{'reject_by_annotation': True,
 'event_id': {'2': 2, '3': 3, '5': 5},
 'selection': array([ 24,  64,  65,  66,  68,  70, 102, 163, 165, 167, 172, 174]),
 'events': array([[ 195000,       0,       2],
        [ 495000,       0,       3],
        [ 502500,       0,       3],
        [ 510000,       0,       3],
        [ 525000,       0,       3],
        [ 540000,       0,       3],
        [ 780000,       0,       3],
        [1237500,       0,       2],
        [1252500,       0,       5],
        [1267500,       0,       2],
        [1305000,       0,       2],
        [1320000,       0,       2]]),
 '_metadata': None,
 'drop_log': (('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGNORED',),
  ('IGN

In [50]:
fs = 250
SETTINGS_B = { 'method' : 'welch', 'average' : 'mean', 'fmin' :30 , 'fmax'  : 45}
electrode  = 'E257'


In [53]:
N2N3_PSD = T_N2N3.compute_psd(**SETTINGS_B, n_fft=fs*5) #N3
N2N1_PSD = T_N2N1.compute_psd(**SETTINGS_B, n_fft=fs*5) #N1

Using data from preloaded Raw for 12 events and 15001 original time points ...
Effective window size : 5.000 (s)
Using data from preloaded Raw for 12 events and 15001 original time points ...
0 bad epochs dropped
Effective window size : 5.000 (s)


In [54]:
N2N3_2 = N2N3_PSD.crop(tmin=0, tmax=5)

AttributeError: 'EpochsSpectrum' object has no attribute 'crop'

In [57]:
N2N3_PSD.crop()


AttributeError: 'EpochsSpectrum' object has no attribute 'crop'